In [5]:
%run "ZeroMQ Python Connector".py
%load_ext autoreload
%autoreload

In [6]:
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = DWX_ZeroMQ_Connector()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [23]:
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(days=100)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1440,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))

{'_action': 'DATA', '_data': {'2019.02.12 00:00': 1.28899, '2019.02.13 00:00': 1.28448, '2019.02.14 00:00': 1.27982, '2019.02.15 00:00': 1.28882, '2019.02.18 00:00': 1.29223, '2019.02.19 00:00': 1.30609, '2019.02.20 00:00': 1.30488, '2019.02.21 00:00': 1.30371, '2019.02.22 00:00': 1.3054, '2019.02.25 00:00': 1.30951, '2019.02.26 00:00': 1.32491, '2019.02.27 00:00': 1.33078, '2019.02.28 00:00': 1.32594, '2019.03.01 00:00': 1.32107, '2019.03.04 00:00': 1.31778, '2019.03.05 00:00': 1.31764, '2019.03.06 00:00': 1.31689, '2019.03.07 00:00': 1.30804, '2019.03.08 00:00': 1.30181, '2019.03.11 00:00': 1.31457, '2019.03.12 00:00': 1.307, '2019.03.13 00:00': 1.3335, '2019.03.14 00:00': 1.32401, '2019.03.15 00:00': 1.32926, '2019.03.18 00:00': 1.32522, '2019.03.19 00:00': 1.32637, '2019.03.20 00:00': 1.31898, '2019.03.21 00:00': 1.31043, '2019.03.22 00:00': 1.32094, '2019.03.25 00:00': 1.31958, '2019.03.26 00:00': 1.32017, '2019.03.27 00:00': 1.31854, '2019.03.28 00:00': 1.3038, '2019.03.29 00:00'

In [24]:
response = Series(connector._get_response_())
responseData = Series(response["_data"])
print(responseData.to_string())

2019.02.12 00:00    1.28899
2019.02.13 00:00    1.28448
2019.02.14 00:00    1.27982
2019.02.15 00:00    1.28882
2019.02.18 00:00    1.29223
2019.02.19 00:00    1.30609
2019.02.20 00:00    1.30488
2019.02.21 00:00    1.30371
2019.02.22 00:00    1.30540
2019.02.25 00:00    1.30951
2019.02.26 00:00    1.32491
2019.02.27 00:00    1.33078
2019.02.28 00:00    1.32594
2019.03.01 00:00    1.32107
2019.03.04 00:00    1.31778
2019.03.05 00:00    1.31764
2019.03.06 00:00    1.31689
2019.03.07 00:00    1.30804
2019.03.08 00:00    1.30181
2019.03.11 00:00    1.31457
2019.03.12 00:00    1.30700
2019.03.13 00:00    1.33350
2019.03.14 00:00    1.32401
2019.03.15 00:00    1.32926
2019.03.18 00:00    1.32522
2019.03.19 00:00    1.32637
2019.03.20 00:00    1.31898
2019.03.21 00:00    1.31043
2019.03.22 00:00    1.32094
2019.03.25 00:00    1.31958
2019.03.26 00:00    1.32017
2019.03.27 00:00    1.31854
2019.03.28 00:00    1.30380
2019.03.29 00:00    1.30292
2019.04.01 00:00    1.31035
2019.04.02 00:00    